# ObiTools

In [1]:
cd ../Desktop/School/FISH546_Git/Sam-Metabarcoding/Data/Bat_Data

/Users/tracykreling/Desktop/School/FISH546_Git/Sam-Metabarcoding/Data/Bat_Data


In [63]:
!head bat_ngs_atom.txt

#exp  sample  forward_primer  reverse_primer	extra_info
bat 1_S1_L001 AGATATTGGAACWTTATATTTTATTTTTGG  WACTAATCAATTWCCAAATCCTCC	F


### Import the data

In [15]:
!obi import --fastq-input 2_S2_L001_R1_001.fastq bat/reads1

2021-02-28 10:52:49,246 [import : INFO ]  obi import: imports an object (file(s), obiview, taxonomy...) into a DMS
2021-02-28 10:52:49,259 [import : INFO ]  Opened file: 2_S2_L001_R1_001.fastq
2021-02-28 10:52:49,404 [import : INFO ]  Importing 64274 entries
 100.0 % |##################################################/] remain : 00:00:00
2021-02-28 10:52:51,932 [import : INFO ]  Imported 64274 entries
2021-02-28 10:52:52,085 [import : INFO ]  Done.


In [16]:
!obi import --fastq-input 2_S2_L001_R2_001.fastq bat/reads2

2021-02-28 10:52:52,442 [import : INFO ]  obi import: imports an object (file(s), obiview, taxonomy...) into a DMS
2021-02-28 10:52:52,455 [import : INFO ]  Opened file: 2_S2_L001_R2_001.fastq
2021-02-28 10:52:52,588 [import : INFO ]  Importing 64274 entries
 100.0 % |##################################################/] remain : 00:00:00
2021-02-28 10:52:55,168 [import : INFO ]  Imported 64274 entries
2021-02-28 10:52:55,297 [import : INFO ]  Done.


In [64]:
!obi import --ngsfilter bat_ngs_atom.txt bat/ngsfile

2021-03-01 11:15:52,587 [import : INFO ]  obi import: imports an object (file(s), obiview, taxonomy...) into a DMS
2021-03-01 11:15:52,594 [import : INFO ]  Opened file: bat_ngs_atom.txt
2021-03-01 11:15:52,610 [import : INFO ]  Importing 2 entries
  50.0 % |#########################-                         ] remain : 00:00:00
2021-03-01 11:15:53,912 [import : INFO ]  Imported 1 entries
2021-03-01 11:15:54,046 [import : INFO ]  Done.


### Check what is in the file structure

In [31]:
!obi ls bat

2021-02-28 16:19:34,829 [ls : INFO ]  obi ls
# DMS name: bat
# Views:
  # aligned_reads: Date created: Sun Feb 28 10:53:21 2021 ; Line count: 64274
  # reads1: Date created: Sun Feb 28 10:52:49 2021 ; Line count: 64274
  # ngsfile: Date created: Sun Feb 28 16:19:29 2021 ; Line count: 2
  # good_sequences: Date created: Sun Feb 28 10:53:39 2021 ; Line count: 64267
  # reads2: Date created: Sun Feb 28 10:52:52 2021 ; Line count: 64274
# Taxonomies:



### Recover the full sequeunces from the partial foward and reverse reads

In [33]:
! obi alignpairedend -R bat/reads2 bat/reads1 bat/aligned_reads

2021-02-28 16:20:07,611 [alignpairedend : INFO ]  obi alignpairedend
 100.0 % |##################################################-] remain : 00:00:00
2021-02-28 16:20:16,427 [alignpairedend : INFO ]  Done.


### Remove unaligned sequence records

In [34]:
!obi grep -a mode:alignment bat/aligned_reads bat/good_sequences

2021-02-28 16:20:17,633 [grep : INFO ]  obi grep
 100.0 % |##################################################/] remain : 00:00:00
2021-02-28 16:20:18,334 [grep : INFO ]  Grepped 64267 entries
2021-02-28 16:20:18,386 [grep : INFO ]  Done.


### Export aligned reads

In [35]:
!obi alignpairedend -R bat/reads2 bat/reads1 - > aligned_reads.fastq

2021-02-28 16:21:02,857 [alignpairedend : INFO ]  obi alignpairedend
 100.0 % |##################################################\] remain : 00:00:00
2021-02-28 16:21:10,252 [alignpairedend : INFO ]  Printing to output...
 100.0 % |##################################################/] remain : 00:00:00
2021-02-28 16:21:13,054 [alignpairedend : INFO ]  Done.


### Assign each sequence record to the corresponding sample/marker combo

Note that the ngs filter must have the F in the 'extra' column or it will not work (thanks Sam for figuring that out!)

In [65]:
!obi ngsfilter -t bat/ngsfile -u bat/unidentified_sequences --no-tags bat/good_sequences bat/identified_sequences

2021-03-01 11:16:02,956 [ngsfilter : INFO ]  obi ngsfilter
 100.0 % |#################################################/ ] remain : 00:00:00
2021-03-01 11:16:37,526 [ngsfilter : INFO ]  Done.


### Dereplicate

NOTE: was run in command line starting at this step, so these commands have not been run

Dereplication allows us to limi computation time by identifying unique sequences and removing replicates while counting how many replicates there were

In [ ]:
!obi uniq bat/identified_sequences bat/dereplicated_sequences

### Denoise

Remove 'useless metadata' and keep only the COUNT and merged_sample (count by sample) tags

In [ ]:
!obi annotate -k COUNT -k MERGED_sample bat/dereplicated_sequences bat/cleaned_metadata_sequences

Keep only the sequences having a count greater or equal to 10 and a length shoert than 80bp

In [ ]:
!obi grep -p "len(sequence)>=80 and sequence['COUNT']>=10" bat/cleaned_metadata_sequences bat/denoised_sequences

Clean the sequences from PCR/sequencing errors (sequence variants)

In [ ]:
!obi clean -s MERGED_sample -r 0.05 -H bat/denoised_sequences bat/cleaned_sequences

### Export Cleaned Sequences

NOTE: may need to remove the '-s MERGED_sample' based on dataset

In [ ]:
 !obi clean -s MERGED_sample -r 0.05 -H bat/denoised_sequences - > cleaned_sequences.fasta